In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# torch.cuda.is_available()

In [2]:
class MyNet(nn.Module):
    """
    多层感知机
    """
    def __init__(self, input_size, hidden_size, output_size):
        super(MyNet, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu1 = torch.nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        z1 = self.linear1(x)
        a1 = self.relu1(z1)
        y_pred = self.linear2(a1)
        return y_pred

In [15]:
def load_train_data(in_name):
    """
    加载训练数据
    """
    x_data = []
    y_data = []
    for line in open(in_name):
        label, vec = line.strip().split('\t')
        y_i = np.zeros(5)
        y_i[int(label)] = 1
        y_data.append(y_i)
        x_data.append(np.array([float(v) for v in vec.split(',')]))
    return x_data, y_data

x_data, y_data = load_train_data('data/train/train_data_ACL-20180712.txt')
x_data = torch.tensor(x_data, dtype=torch.float)
y_data = torch.tensor(y_data, dtype=torch.float)
print(x_data.size(), y_data.size())

torch.Size([11991, 300]) torch.Size([11991, 5])


In [ ]:
# 设置参数
input_size, hidden_size, output_size = 300, 100, 5
learning_rate = 1e-5
EPOCH = 5000

# 定义函数
mod = MyNet(input_size, hidden_size, output_size)

# 定义损失函数
loss_fn = nn.MSELoss(size_average=False)

# 优化器
optimizer = torch.optim.SGD(mod.parameters(), lr=learning_rate)

In [38]:
## 开始训练 ##
for t in range(EPOCH):
    
    # 向前传播
    y_pred = mod(x_data)
    
    # 计算损失
    loss = loss_fn(y_pred, y_data)
    # 显示损失
    if t % 500 == 0:
        print(loss.data.item())
    
    # 在我们进行梯度更新之前，
    # 先使用optimier对象提供的清除已经积累的梯度。
    optimizer.zero_grad()
    
    # 计算梯度
    loss.backward()
    
    # 更新梯度
    optimizer.step()

11615.4599609375
8750.8349609375
8617.62109375
8526.04296875
8466.537109375
8412.2197265625
8342.642578125
8262.72265625
8200.2548828125
8079.6328125
